In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
# here we are working on Tensorflow version 2.1.0 so we need to write tensorflow.keras.
#keras is in built function in Tensorflow.
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image

In [4]:
train_location = "/content/drive/MyDrive/Colab Notebooks/Diabetic Retenopathy/Dataset"
test_location = "/content/drive/MyDrive/Colab Notebooks/Diabetic Retenopathy/Dataset"
filepath = '/content/drive/MyDrive/Colab Notebooks/Diabetic Retenopathy/Model/model1_VGG16.keras'

In [5]:
from tensorflow.keras.models import load_model
Detection=load_model(filepath)

In [6]:
img_size=224
batch_size=10
num_class=2

In [7]:
# Complete Dataset images can be loaded using ImageDataGenerator function

datagen_train=ImageDataGenerator(horizontal_flip=True)
train_generator=datagen_train.flow_from_directory(train_location,target_size=(img_size,img_size),batch_size=batch_size,class_mode='categorical',shuffle=True)

datagen_test=ImageDataGenerator(horizontal_flip=True)
validation_generator=datagen_test.flow_from_directory(test_location,target_size=(img_size,img_size),batch_size=batch_size,class_mode='categorical',shuffle=True)


Found 82 images belonging to 2 classes.
Found 82 images belonging to 2 classes.


In [8]:
classes=train_generator.class_indices
classes

{'Exudate': 0, 'Healthy': 1}

In [9]:
category=[]
for i in classes:
          category.append(i)

category

['Exudate', 'Healthy']

In [10]:
from skimage import io
import os
from tensorflow.keras.preprocessing import image

image_directory=test_location
dataset = []
predict_result=[]
label =[]


my_folders = os.listdir(image_directory)
for i, folder_name in enumerate(my_folders):
    #print(str(i)+': ' +folder_name)

    loc=0
    for j in category:
      if(j==folder_name):
        lab=loc
      loc+=1
    lab

    my_images = os.listdir(image_directory+'/'+folder_name)
    for j,image_name in enumerate(my_images):
      #print(str(j)+': ' +image_name)

      file_name = image_directory+'/'+folder_name + '/' + image_name

      test_img=image.load_img(file_name,target_size=(img_size,img_size))
      test_img=image.img_to_array(test_img)
      test_img=np.expand_dims(test_img,axis=0)
      result=Detection.predict(test_img)

      label.append(lab)
      predict_result.append(result.argmax())

      print(f'{folder_name} : {str(lab)} , Predicted : {result.argmax()}  ')




1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 888ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 942ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step
Healthy : 1 , Predicted : 0  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
Healthy : 1 , Predicted : 0  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
Healthy : 1 , Predicted : 0  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
Healthy : 1 , Predicted : 1  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step
H

In [11]:
correct=0
Wrong=0


for i in range(len(label)):
  if(predict_result[i]==label[i]):
    correct+=1
  else:
    Wrong+=1

print(f'correct: {correct} , Wrong: {Wrong}')




correct: 73 , Wrong: 9


In [12]:
Acc= correct/(correct+Wrong)
Acc*100

89.02439024390245

In [13]:

y_test = label
y_pred = predict_result

In [14]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('Test set accuracy: ',metrics.accuracy_score(y_test, y_pred))

[[45  2]
 [ 7 28]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.91        47
           1       0.93      0.80      0.86        35

    accuracy                           0.89        82
   macro avg       0.90      0.88      0.89        82
weighted avg       0.89      0.89      0.89        82

Test set accuracy:  0.8902439024390244


In [15]:
confusion = metrics.confusion_matrix(y_test, y_pred)
confusion.ravel()

array([45,  2,  7, 28])

In [16]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.8902439024390244